# SQL Databases
- easy to use from Python
- many different dbs are supported: sqlite3, mysql, postgres, oracle...

In [1]:
# need some names to build a database, so made some random strings
# but they are hard to read...

import random
import string

string.ascii_lowercase

def makeNames(n):
    # chr takes an ascii code and returns the letter in a string
    n = ''.join([random.choice(string.ascii_lowercase) for j in range(n)])
    return n.capitalize()
    
[makeNames(4) for j in range(10)]

['Ekau',
 'Zczp',
 'Eajf',
 'Hfdi',
 'Cxic',
 'Lgwo',
 'Alxy',
 'Evsn',
 'Okcj',
 'Zikg']

In [2]:
# nltk corpus has 8,000 real names!
# get some short ones

import random
import nltk

def names(wcnt, wlen):
    names = [w for w in nltk.corpus.names.words() if len(w) == wlen]
    # names are in alphabetical order
    # random doesn't have a 'random subset' routine
    # so do a shuffle 
    random.shuffle(names)
    # then just take random names from the front
    return names[:wcnt]

firsts = names(10, 4)
lasts = names(10, 5)

[firsts, lasts]

[['Pace',
  'Olin',
  'Lyda',
  'Lynn',
  'Rana',
  'Otto',
  'Kane',
  'Elyn',
  'Rees',
  'Nara'],
 ['Mohan',
  'Stern',
  'Debee',
  'Irita',
  'Sheff',
  'Cosmo',
  'Bevvy',
  'Wayne',
  'Adger',
  'Sarah']]

In [3]:
# sqlite stores data in one file
# delete old db if present

import sqlite3
import os
import random

dbf = '/tmp/3.db'
os.remove(dbf)

In [4]:
# make a db connection and get a cursor

con = sqlite3.connect(dbf)
cur = con.cursor()

# Create two tables
- grade(uni, course, grade)
- student(first, last, uni)

In [5]:
cur.execute('create table grade (uni text, course text, grade real)')

In [6]:
cur.execute('create table student (first text, last text, uni text)')

# Build student table

In [7]:
# first, last

fl = [ [firsts[k], lasts[k]] for k in range(10)]
fl

[['Pace', 'Mohan'],
 ['Olin', 'Stern'],
 ['Lyda', 'Debee'],
 ['Lynn', 'Irita'],
 ['Rana', 'Sheff'],
 ['Otto', 'Cosmo'],
 ['Kane', 'Bevvy'],
 ['Elyn', 'Wayne'],
 ['Rees', 'Adger'],
 ['Nara', 'Sarah']]

In [8]:
# make an uni
# student = [first, last, uni]

students = [ [f, l, (f[0] + l[0] + str(random.randint(1000,9999))).lower()] 
            for f,l in fl]
students

[['Pace', 'Mohan', 'pm5377'],
 ['Olin', 'Stern', 'os7127'],
 ['Lyda', 'Debee', 'ld3799'],
 ['Lynn', 'Irita', 'li9704'],
 ['Rana', 'Sheff', 'rs9563'],
 ['Otto', 'Cosmo', 'oc7279'],
 ['Kane', 'Bevvy', 'kb4481'],
 ['Elyn', 'Wayne', 'ew9191'],
 ['Rees', 'Adger', 'ra9305'],
 ['Nara', 'Sarah', 'ns5859']]

In [9]:
# insert above list into db

# 'execute' inserts one row at a time
# expects one row of data
cur.execute('insert into student values(?, ?, ?)', students[0])

# 'executemany' inserts multiple rows at once
# expects list of rows, executes the statement for each row
cur.executemany('insert into student values(?, ?, ?)', students[1:])

'''
# the above two statements are equivalent to 
for student in students:
    cur.execute('insert into student values(?, ?, ?)', student)

# or 
cur.executemany('insert into student values(?, ?, ?)', students)

executemany is typically more efficient 
'''

None

In [10]:
# the select returns a generator

sg = cur.execute('select first, last, uni from student')
sg

In [11]:
rows = list(sg)
rows

[('Pace', 'Mohan', 'pm5377'),
 ('Olin', 'Stern', 'os7127'),
 ('Lyda', 'Debee', 'ld3799'),
 ('Lynn', 'Irita', 'li9704'),
 ('Rana', 'Sheff', 'rs9563'),
 ('Otto', 'Cosmo', 'oc7279'),
 ('Kane', 'Bevvy', 'kb4481'),
 ('Elyn', 'Wayne', 'ew9191'),
 ('Rees', 'Adger', 'ra9305'),
 ('Nara', 'Sarah', 'ns5859')]

In [12]:
# pull the uni out of the tuple 

unis = [t[0] for t in cur.execute('select uni from student')]
unis

['pm5377',
 'os7127',
 'ld3799',
 'li9704',
 'rs9563',
 'oc7279',
 'kb4481',
 'ew9191',
 'ra9305',
 'ns5859']

In [13]:
# classes 

classes = ['COMSW {}'.format(random.randint(1000, 9999)) for j in range(5)]
classes

['COMSW 8591', 'COMSW 2830', 'COMSW 3875', 'COMSW 8058', 'COMSW 4921']

In [14]:
# students take finals

grades = [ [u , random.choice(classes), 70 + 30 * random.random() ] for u in unis ]
grades

[['pm5377', 'COMSW 4921', 95.92742946895079],
 ['os7127', 'COMSW 8591', 88.94923956134704],
 ['ld3799', 'COMSW 3875', 89.12528801822131],
 ['li9704', 'COMSW 4921', 99.3032131590247],
 ['rs9563', 'COMSW 4921', 99.46685234575497],
 ['oc7279', 'COMSW 8591', 72.68857034516523],
 ['kb4481', 'COMSW 4921', 91.48971627211276],
 ['ew9191', 'COMSW 3875', 91.74460096867148],
 ['ra9305', 'COMSW 8058', 82.05360968104378],
 ['ns5859', 'COMSW 3875', 79.11870868404564]]

In [15]:
# executemany inserts multiple rows at once 

cur.executemany('insert into grade values(?, ?, ?)', grades)

In [16]:
# join the two tables on the uni field

list(cur.execute('select first,last,student.uni, \
                 grade from student,grade where student.uni = grade.uni'))

[('Pace', 'Mohan', 'pm5377', 95.92742946895079),
 ('Olin', 'Stern', 'os7127', 88.94923956134704),
 ('Lyda', 'Debee', 'ld3799', 89.12528801822131),
 ('Lynn', 'Irita', 'li9704', 99.3032131590247),
 ('Rana', 'Sheff', 'rs9563', 99.46685234575497),
 ('Otto', 'Cosmo', 'oc7279', 72.68857034516523),
 ('Kane', 'Bevvy', 'kb4481', 91.48971627211276),
 ('Elyn', 'Wayne', 'ew9191', 91.74460096867148),
 ('Rees', 'Adger', 'ra9305', 82.05360968104378),
 ('Nara', 'Sarah', 'ns5859', 79.11870868404564)]

In [17]:
# add a filter term

q = 'select last,first,grade from student,grade \
 where grade>80 and student.uni = grade.uni \
 order by last'
list(cur.execute(q))

[('Adger', 'Rees', 82.05360968104378),
 ('Bevvy', 'Kane', 91.48971627211276),
 ('Debee', 'Lyda', 89.12528801822131),
 ('Irita', 'Lynn', 99.3032131590247),
 ('Mohan', 'Pace', 95.92742946895079),
 ('Sheff', 'Rana', 99.46685234575497),
 ('Stern', 'Olin', 88.94923956134704),
 ('Wayne', 'Elyn', 91.74460096867148)]

In [18]:
# always commit and close the connection

con.commit()
con.close()

In [19]:
# data is persisted on disk, can read again
# normally would use a with statement to automatically close

res=None

with sqlite3.connect(dbf) as con:
    cur = con.cursor()
    res = list(cur.execute(q))

res

[('Adger', 'Rees', 82.05360968104378),
 ('Bevvy', 'Kane', 91.48971627211276),
 ('Debee', 'Lyda', 89.12528801822131),
 ('Irita', 'Lynn', 99.3032131590247),
 ('Mohan', 'Pace', 95.92742946895079),
 ('Sheff', 'Rana', 99.46685234575497),
 ('Stern', 'Olin', 88.94923956134704),
 ('Wayne', 'Elyn', 91.74460096867148)]

# Object Relational Mappers
- maps objects into a relational database
- somewhat complex but very useful
- best known one is [SQLAlchemy](http://www.sqlalchemy.org)
- [peewee](http://docs.peewee-orm.com/en/latest/) is a simple one
- [hibernate](http://hibernate.org) was a pioneering and hugely successful ORM for Java

# NoSQL databases
- provide less functionality than SQL, but are more efficient and scale better
- [mongodb](https://www.mongodb.org) is a popular one
- [PyMongo](https://docs.mongodb.org/getting-started/python/client/) is the python driver for mongodb
